## Player Comparison Model Using Neural Networks

- This is the first iteration to see how well this is going to work for the data that I have from previous and current seasons
- I am planning to base this off of a few different fields depending on the position. Currently I'm not too sure if I will have to do a model depending on the position of the player as different stats matter more for different positions

In [5]:
# The first step is combining the previous seasons data so I can train the model based off of a singular csv file.

import pandas as pd
import numpy as np
import sklearn

# Load the data
df = pd.read_csv('Total-Season-Player-Stats.csv')

print(df.head())

  Rk               Player   Nation    Pos           Squad Age  Born  MP  \
0  1  Patrick van Aanholt   nl NED     DF  Crystal Palace  26  1990  28   
1  2       Rolando Aarons  eng ENG  MF,FW   Newcastle Utd  21  1995   4   
2  3        Tammy Abraham  eng ENG     FW    Swansea City  19  1997  31   
3  4         Charlie Adam  sct SCO     MF      Stoke City  31  1985  11   
4  5               Adrián   es ESP     GK        West Ham  30  1987  19   

  Starts   Min  ... Ast.1 G+A.1 G-PK.1 G+A-PK  xG.1 xAG.1 xG+xAG npxG.1  \
0     25  2184  ...  0.04  0.25   0.21   0.25  0.13  0.09   0.21   0.13   
1      1   139  ...  0.00  0.00   0.00   0.00  0.04  0.00   0.04   0.04   
2     15  1726  ...  0.05  0.31   0.26   0.31  0.35  0.08   0.44   0.35   
3      5   411  ...  0.00  0.00   0.00   0.00  0.36  0.27   0.63   0.19   
4     19  1710  ...  0.00  0.00   0.00   0.00  0.00  0.00   0.00   0.00   

  npxG+xAG.1  Matches  
0       0.21  Matches  
1       0.04  Matches  
2       0.44  Matches  
3 

In [13]:
# Load the dataset (replace with your actual file path)
df = pd.read_csv('Total-Season-Player-Stats.csv')

# Ensure the first column is treated as a string and remove any leading/trailing spaces
df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip()

# Filter out rows starting with 'Rk' (excluding the first row)
filtered_df = df.iloc[1:].copy()
filtered_df = filtered_df[~filtered_df.iloc[:, 0].str.startswith('Rk', na=False)]

# Re-add the first row (index 0) and reset index
df = pd.concat([df.iloc[[0]], filtered_df], ignore_index=True)

# Save the cleaned data
df.to_csv('Total-Season-Player-Stats-Filtered.csv', index=False)

This is just to remove some of the unneeded headers that were in the CSV as this CSV file was a combination of the past few years of data. I had ChatGPT partially help with this part as I was unsure why my initial method of doing this wasn't working.

<br>

I want to try a random forest model for this as I know that they work with smaller datasets and they should be good as it is a binary result.

The next issue that I am running into for this model is that I don't have a clear target variable, I am thinking to make some sort of formula that will combine and rank the most important stats for defenders, midfielders and attackers. The user will be able to select a position and then this will give them a second dropdown of players where they can select 2 and it will use this formula as the target variable for the model. I have to do different models here as different stats are going to be important for different positions.

In [14]:
#  I am going to start by working on the attackers model first. I will filter the data to only include attackers and then I will train the model based off of that data.

fw_df = df[df['Pos'].str.startswith("FW", na=False)] # I am doing this as some players have multiple positions but their main one is always listed first so I can just use the first two characters to filter
print(fw_df.head())

    Rk           Player   Nation    Pos            Squad Age  Born  MP Starts  \
2    3    Tammy Abraham  eng ENG     FW     Swansea City  19  1997  31     15   
6    7      Benik Afobe   cd COD     FW      Bournemouth  24  1993  17      5   
7    8    Sergio Agüero   ar ARG     FW  Manchester City  29  1988  25     22   
17  18   Nordin Amrabat   ma MAR     FW          Watford  30  1987   3      3   
18  19  Michail Antonio   jm JAM  FW,MF         West Ham  27  1990  21     16   

     Min  ... Ast.1 G+A.1 G-PK.1 G+A-PK  xG.1 xAG.1 xG+xAG npxG.1 npxG+xAG.1  \
2   1726  ...  0.05  0.31   0.26   0.31  0.35  0.08   0.44   0.35       0.44   
6    611  ...  0.00  0.00   0.00   0.00  0.27  0.02   0.30   0.27       0.30   
7   1963  ...  0.28  1.24   0.78   1.05  0.75  0.26   1.00   0.60       0.86   
17   198  ...  0.00  0.00   0.00   0.00  0.02  0.06   0.09   0.02       0.09   
18  1354  ...  0.07  0.27   0.20   0.27  0.16  0.07   0.23   0.16       0.23   

    Matches  
2   Matches  
6   

My next step here is to create a formula that I will apply to each player in the dataset, this is because there isn't one specific statistic that I want my model to judge a player by its more of a culmination of a few of them, for example:

#### Attackers:
1. XG per 90
2. Goals per 90
3. NPXG + XAG per 90
4. Assists per 90

#### Midfielders:
1. Progressive passes
2. XAG per 90
3. XG per 90
4. Progressive carries

#### Defenders:
1. Clearances
2. NPXG + XAG per 90
3. Blocks
4. Interceptions
<br>

For most of these stats we already have the data and it is fairly straight forward, however juding a defenders performances can be tough as they are more nuanced stats, I believe that these are the 4 main ones and I have researched online for what statistics other people use for judging defenders and midfielders from places such as these:
https://bleacherreport.com/articles/1722602-which-stats-are-most-important-for-measuring-defenders
https://www.backheeled.com/what-stats-are-used-to-evaluate-central-midfielders/#:~:text=Things%20like%20pressures%2C%20tackles%2C%20and,doesn't%20have%20the%20ball.

For the defenders statistics I am going to have to pull in a new table of data as the stats that I want to use aren't in the current table, to do this I am going to repeat the process that I used to import the previous seasons data but Instead I am going to just get the defensive actions data. This is being done in old-defender-data.ipynb.